In [1]:
matrix?

Signature:      matrix(*args, **kwds)
Call signature: matrix(*args, **kwargs)
Type:           cython_function_or_method
String form:    <cyfunction matrix at 0x148506a80>
File:           /private/var/tmp/sage-10.6-current/local/var/lib/sage/venv-python3.12.5/lib/python3.12/site-packages/sage/matrix/constructor.pyx
Docstring:     
   Create a matrix.

   This implements the "matrix" constructor:

      sage: matrix([[1,2],[3,4]])
      [1 2]
      [3 4]

   It also contains methods to create special types of matrices, see
   "matrix.[tab]" for more options. For example:

      sage: matrix.identity(2)
      [1 0]
      [0 1]

   INPUT:

   The "matrix()" command takes the entries of a matrix, optionally
   preceded by a ring and the dimensions of the matrix, and returns a
   matrix.

   The entries of a matrix can be specified as a flat list of
   elements, a list of lists (i.e., a list of rows), a list of Sage
   vectors, a callable object, or a dictionary having positions as
   keys a

In [1]:
def structured_matrix(n, d_list):
    """
    Constructs the n x n matrix A with:
    - A[i,j] = d_{i-j} for i >= j (lower diagonals, including main)
    - A[i,i+1] = i+1 for i = 0 to n-2 (first superdiagonal)
    - all other entries zero

    Input:
        n      : integer >= 1
        d_list : list of length n, with values [d_0, d_1, ..., d_{n-1}]

    Output:
        A : n x n matrix over the parent of d_list[0]
    """
    if len(d_list) != n:
        raise ValueError("d_list must have length n")
    
    # Determine base ring from d_list
    R = parent(d_list[0])
    A = Matrix(R, n, n, 0)  # initialize to zero matrix

    for i in range(n):
        for j in range(n):
            if i >= j:
                A[i, j] = d_list[i - j]  # subdiagonals (including main)
            elif j == i + 1:
                A[i, j] = i + 1          # superdiagonal entries 1, 2, ..., n-1

    return A


In [2]:
d_list=[5,7]
n=2
structured_matrix(n, d_list)

[5 1]
[7 5]

In [ ]:
def structured_determinant(n, d_list):
    """
    Compute det(A_n) for the matrix defined by structured_matrix(n, d_list)
    using a recurrence:
    
        D_n = d_0 D_{n-1} - (n-1)d_1 D_{n-2} + (n-2)d_2 D_{n-3} - ...
    
    Input:
        n      : integer >= 1
        d_list : list of length >= n, values [d_0, d_1, ..., d_{n-1}]
                 must be elements of a symbolic ring or field

    Output:
        D_n : symbolic expression for determinant
    """
    if len(d_list) < n:
        raise ValueError("d_list must be at least length n")
    
    # Use SR for symbolic work
    D = [SR(1)]  # D_0 = 1
    
    for k in range(1, n + 1):
        term = d_list[0] * D[k - 1]
        for j in range(1, k):
            sign = (-1)**j
            coeff = (k - j)
            term += sign * coeff * d_list[j] * D[k - j - 1]
        D.append(term)

    return D[n]


In [ ]:
def structured_matrix(n, d_list):
    """
    Construct the n x n matrix A with:
      - entries on and below the main diagonal given by d_list (d_0 on main diagonal,
        d_1 on first subdiagonal, ..., d_{n-1} on last subdiagonal),
      - entries on the first superdiagonal equal to 1, 2, ..., n-1,
      - all other entries zero.

    INPUT:
      - n : positive integer
      - d_list : list of length n with diagonal values [d_0, d_1, ..., d_{n-1}]
    
    OUTPUT:
      - n x n matrix over parent ring of d_list elements
    """
    if len(d_list) != n:
        raise ValueError("d_list length must be equal to n")
    
    R = parent(d_list[0])
    A = Matrix(R, n, n, 0)  # initialize zero matrix
    
    for i in range(n):
        for j in range(n):
            if i >= j:
                A[i, j] = d_list[i - j]  # on and below main diagonal
            elif j == i + 1:
                A[i, j] = i + 1          # superdiagonal entries: 1,2,...,n-1
            else:
                A[i, j] = 0              # zeros elsewhere
    
    return A


In [1]:
# Importing necessary library
A = Matrix([[4, -2], [1, 1]])

# Find eigenvalues of matrix A
eigenvalues = A.eigenvalues()
print("Eigenvalues:", eigenvalues)


Eigenvalues: [3, 2]


In [5]:
# Define the matrix
A = Matrix([[2, 1], 
            [1, 2]])

# Compute the eigenvectors
eigenvectors = A.eigenvectors_right()

# Display the eigenvectors
for eigval, eigvecs, multiplicity in eigenvectors:
    print(f"Eigenvalue: {eigval}")
    print(f"Multiplicity: {multiplicity}")
    print("Eigenvectors:")
    for v in eigvecs:
        print(v)
    print()

Eigenvalue: 3
Multiplicity: 1
Eigenvectors:
(1, 1)

Eigenvalue: 1
Multiplicity: 1
Eigenvectors:
(1, -1)



In [9]:
Rx.<x>= PolynomialRing(QQ)
Pm = PolynomialRing(QQ, 'm')
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)
def H_matrix(h_list):
    n = len(h_list)
    H = Matrix(QQ, n, n)

    # Column 0 (first column): i*h_i for i from 1 to n
    for i in range(n):
        H[i, 0] = (i + 1) * h_list[i]

    # Column 1 and onwards: each pushes down previous column
    for col in range(1, n):
        for row in range(n):
            if row == 0:
                H[row, col] = 0
            else:
                H[row, col] = H[row - 1, col - 1]

    return H

In [4]:
h_list=[10,10,10]
H_matrix(h_list)

[10  0  0]
[20 10  0]
[30 20 10]

In [12]:
h_list=[]
for n in [1,2,3]:
    h_list+=[ramanujanTau(n+1)]
H_matrix(h_list)

[  -24     0     0]
[  504   -24     0]
[-4416   504   -24]

In [13]:
import pickle 
Rx.<x>= PolynomialRing(QQ)
x = var('x')
def H_matrix(h_list):
    n = len(h_list)
    H = Matrix(QQ, n, n)

    # Column 0 (first column): i*h_i for i from 1 to n
    for i in range(n):
        H[i, 0] = (i + 1) * h_list[i]

    # Column 1 and onwards: each pushes down previous column
    for col in range(1, n):
        for row in range(n):
            if row == 0:
                H[row, col] = 0
            else:
                H[row, col] = H[row - 1, col - 1]

    return H

def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)

charpolys=[]

for N in range(1, 10):
    
    inputs = [ramanujanTau(n+1) for j in range(1, N + 1)]
    hm = H_matrix(inputs)
    d = hm.det()
    p = hm.charpoly(x)#.change_ring(SR)
    charpolys+=[(N,p)]
    print("N:",N,"charpoly:",p)
#print("tests:")
#print(tests)
#wfile = open('/Users/barrybrent/data/run23jul25no1.txt','wb') 
#t = pickle.dumps(str(charpolys))
#wfile.write(t)
#wfile.close() 

N: 1 charpoly: x + 1472
N: 2 charpoly: x^2 + 2944*x + 2166784
N: 3 charpoly: x^3 + 4416*x^2 + 6500352*x + 3189506048
N: 4 charpoly: x^4 + 5888*x^3 + 13000704*x^2 + 12758024192*x + 4694952902656
N: 5 charpoly: x^5 + 7360*x^4 + 21667840*x^3 + 31895060480*x^2 + 23474764513280*x + 6910970672709632
N: 6 charpoly: x^6 + 8832*x^5 + 32501760*x^4 + 63790120960*x^3 + 70424293539840*x^2 + 41465824036257792*x + 10172948830228578304
N: 7 charpoly: x^7 + 10304*x^6 + 45502464*x^5 + 111632711680*x^4 + 164323351592960*x^3 + 145130384126902272*x^2 + 71210641811600048128*x + 14974580678096467263488
N: 8 charpoly: x^8 + 11776*x^7 + 60669952*x^6 + 178612338688*x^5 + 328646703185920*x^4 + 387014357671739392*x^3 + 284842567246400192512*x^2 + 119796645424771738107904*x + 22042582758157999811854336
N: 9 charpoly: x^9 + 13248*x^8 + 78004224*x^7 + 267918508032*x^6 + 591564065734656*x^5 + 870782304761413632*x^4 + 854527701739200577536*x^3 + 539084904411472821485568*x^2 + 198383244823421998306689024*x + 3244668182

In [15]:
import pickle 
Rx.<x>= PolynomialRing(QQ)
x = var('x')
def H_matrix(h_list):
    n = len(h_list)
    H = Matrix(QQ, n, n)

    # Column 0 (first column): i*h_i for i from 1 to n
    for i in range(n):
        H[i, 0] = (i + 1) * h_list[i]

    # Column 1 and onwards: each pushes down previous column
    for col in range(1, n):
        for row in range(n):
            if row == 0:
                H[row, col] = 0
            else:
                H[row, col] = H[row - 1, col - 1]

    return H

def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)

charpolys=[]

for N in range(1, 100):
    
    inputs = [ramanujanTau(n+1) for j in range(1, N + 1)]
    hm = H_matrix(inputs)
    d = hm.det()
    p = hm.charpoly(x)#.change_ring(SR)
    charpolys+=[(N,p)]
wfile = open('/Users/barrybrent/data/run28jul25no1.txt','wb') 
t = pickle.dumps(str(charpolys))
wfile.write(t)
wfile.close() 

In [90]:
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)
Rx= PolynomialRing(QQ, 'x')
Pm = PolynomialRing(QQ, 'm')
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def rdr(p,n):
    return QQ.valuation(p)(n)
def sig(j):
    return 24*sigma(j)
rfile = open('/Users/barrybrent/data/run28jul25no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
cffx0=[]
data=[]
for k in range(1,len(s)):
        N = s[k][0]
        for n in range(0,N+1):
            pair=(N,n)
            poly=Rx(s[k][1])
            epc=extendedPolynomialCoefficient(n,poly)
            cff=2^(6*N-6*n)*23^(N-n)
            den=cff*binomial(N,n)
            data+=[epc/den]
print(data)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [91]:
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]
def ramanujanTau(n):
    M = CuspForms(1, 12)
    f = Rx(M.q_expansion_basis(n + 1)[0])
    return f.coefficient(n)
Rx= PolynomialRing(QQ, 'x')
Pm = PolynomialRing(QQ, 'm')
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def rdr(p,n):
    return QQ.valuation(p)(n)
def sig(j):
    return 24*sigma(j)
rfile = open('/Users/barrybrent/data/run28jul25no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
cffx0=[]
data=[]
for k in range(1,len(s)):
        N = s[k][0]
        for n in range(0,N+1):
            pair=(N,n)
            poly=Rx(s[k][1])
            epc=extendedPolynomialCoefficient(n,poly)
            cff=64^(N-n)*23^(N-n)
            den=cff*binomial(N,n)
            data+=[epc/den]
print(data)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [92]:
23*64

1472